# Bigraph-Builder Demo

In [ ]:
from builder import Builder, Process, ProcessTypes

## Initialize the builder

In [ ]:
from biosimulator_processes import CORE

In [ ]:
b = Builder(core=CORE)

### register new types

In [ ]:
b.register_type(
    'default 1', {
        '_inherit': 'float',
        '_default': 1.0})

## Register processes

### list built-in processes

In [ ]:
b.list_processes()

### register process with process class

In [ ]:
from process_bigraph.experiments.minimal_gillespie import GillespieEvent
b.register_process('GillespieEvent', GillespieEvent)

### register process by address
currently only supports local addresses, but the plan is to support remote addresses and different protocols

In [ ]:
b.register_process(
    'GillespieInterval',
    address='process_bigraph.experiments.minimal_gillespie.GillespieInterval',  
)

### register with decorator

In [ ]:
@b.register_process('toy')
class Toy(Process):
    config_schema = {
        'A': 'float',
        'B': 'float'}

    def __init__(self, config, core):
        super().__init__(config, core)

    def schema(self):
        return {
            'inputs': {
                'A': 'float',
                'B': 'float'},
            'outputs': {
                'C': 'float'}}

    def update(self, state, interval):
        update = {'C': state['A'] + state['B']}
        return update


### list registered processes

In [ ]:
b.list_processes()

## Add processes to the bigraph

### add event_process

In [ ]:
from biosimulator_processes.process_types import  MODEL_TYPE

MODEL_TYPE

In [ ]:
species_changes = ModelChange(config={
    'A': {
        'initial_expression': 'A -> B'
    }
})

global_params_changes = ModelChange(config={
    'Bol': {
        'unit': 'ml'
    }
})

model_changes = ModelChanges(species_changes=species_changes, global_parameter_changes=global_params_changes)


model = SedModel(
    model_id='BIO232423',
    model_source='uri:2424',
    model_language='sbml',
    model_name='Glyc',
    model_changes=model_changes
)


In [ ]:
model.model_dump()

In [ ]:
copasi_config = 
b['copasi_A'].add_process(
    name='CopasiProcess',
    kdeg=1.0,  # kwargs fill parameters in the config
)

In [ ]:
# visualize shows the process with its disconnected ports
b.visualize()

### print ports

In [ ]:
b['event_process'].interface(True)

### connect ports using connect_all
`Builder.connect_all` connects ports to stores of the same name.

In [ ]:
b.connect_all(append_to_store_name='_store')
b

In [ ]:
b.visualize()

### add interval process to the config

In [ ]:
b['interval_process'].add_process(
    name='GillespieInterval',
)

In [ ]:
b.visualize()

### connect port to specific target

In [ ]:
# to connect a port in a more targeted way, use connect and specify the port and its target path
b['interval_process'].connect(port='interval', target=['event_process', 'interval']) 

# the remaining ports can connect_all
b.connect_all() 

In [ ]:
b.visualize()

### check current Builder config

In [ ]:
b

## Update the initial state

In [ ]:
initial_state = {
    'DNA_store': {
        'A gene': 2.0,
        'B gene': 1.0},
}
b.update(initial_state)

## Generate composite from builder config and simulate

In [ ]:
composite = b.generate()
composite.run(10)

In [ ]:
composite

## Retrieve the composite document

In [ ]:
doc = b.document()
doc

### save the document to file

In [ ]:
b.write(filename='toy_bigraph')

### load a composite from document
This document represents the full state of the composite, and so can reproduce the previous composite when loaded into a fresh Builder

In [ ]:
b2 = Builder(core=core, file_path='out/toy_bigraph.json')
b2

In [ ]:
b2.visualize()

In [1]:
from biosimulator_processes.data_model import BasicoModelChanges

CobraProcess registered successfully.
CopasiProcess not available. Error: No module named 'biosimulator_processes.process_types'
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


In [2]:
changes = BasicoModelChanges()

In [3]:
changes

{'species_changes': {'species_name': {'unit': 'maybe[string]',
   'initial_concentration': 'maybe[float]',
   'initial_particle_number': 'maybe[float]',
   'initial_expression': 'maybe[string]',
   'expression': 'maybe[string]'}},
 'global_parameter_changes': {'global_parameter_name': {'initial_value': 'maybe[float]',
   'initial_expression': 'maybe[string]',
   'expression': 'maybe[string]',
   'status': 'maybe[string]',
   'type': 'maybe[string]'}},
 'reaction_changes': {'reaction_name': {'parameters': {'reaction_parameter_name': 'maybe[int]'},
   'reaction_scheme': 'maybe[string]'}}}

In [4]:
type(changes)

biosimulator_processes.data_model.BasicoModelChanges